<a href="https://colab.research.google.com/github/caua-sathler/NEURAL-NETWORKS/blob/main/MLP-PSO-SGD/MLP_PSO_Adam_Ionosphere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [13]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data"
column_names = [f"feature_{i}" for i in range(34)] + ["label"]

ionosphere = pd.read_csv(url, header=None, names=column_names)

ionosphere["label"] = ionosphere["label"].map({"b": 0, "g": 1})

In [16]:
X = ionosphere.drop(columns=['label']).values
y = ionosphere['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [17]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 16)
        self.fc6 = nn.Linear(16, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [21]:
class Particle:
    def __init__(self, model, device):
        self.model = copy.deepcopy(model).to(device)
        self.best_model = copy.deepcopy(model).to(device)

        low, high = -10.0, 10.0
        velocity_scale = 0.1

        self.position = {
            name: torch.rand_like(param).to(device)*(high - low)+low
            for name, param in model.named_parameters()
        }
        self.velocity = {
            name: torch.randn_like(param).to(device)*velocity_scale
            for name, param in model.named_parameters()
        }

        self.best_loss = float('inf')  # melhor perda local
        self.device = device

        self.optimizer = optim.Adam(self.model.parameters(), lr=0.007, weight_decay=1e-5)

    # def calculate_grad(self, x_train, y_train, criterion):
    #     self.model.train()
    #     self.optimizer.zero_grad()

    #     outputs = self.model(x_train)
    #     loss = criterion(outputs, y_train)
    #     loss.backward()
    #     self.optimizer.step()

    #     # _, predicted = torch.max(outputs.data, 1)
    #     # acc = (predicted == y_train).sum().item() / len(x_train)
    #     # return loss.item(), acc*100

    def pso(self, global_best_model, inertia, c1, c2):
        for name, param in self.model.named_parameters():
            # if param.grad is None:
            #     continue

            local_rand = random.random()
            global_rand = random.random()

            # Atualização da velocidade
            self.velocity[name] = (
                inertia*self.velocity[name]
                + c1*local_rand*(self.best_model.state_dict()[name].to(self.device) - param.data)
                + c2*global_rand*(global_best_model.state_dict()[name].to(self.device) - param.data)
            )

            # Atualizar posição
            self.position[name] = param.data + self.velocity[name]
            param.data = self.position[name]

            # particle.calculate_grad(X_train, y_train, criterion)

            # # Adam simplificado
            # m[name] = m[name].to(param.device)
            # v[name] = v[name].to(param.device)

            # m[name] = beta1*m[name] + (1-beta1)*param.grad
            # v[name] = beta2*v[name] + (1-beta2)*(param.grad**2)

            # # Substituição final
            # param.data -= lr * (m[name] / (torch.sqrt(v[name]) + epsilon))
            # # param.data = self.position[name] - lr * param.grad

            # # Refinamento dos pesos com o Adam
            # outputs = particle.model(X_train)
            # loss = criterion(outputs, y_train)
            # loss.backward()
            # particle.optimizer.step()


    def evaluate_weights(self, x, y, criterion):
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(x)
            loss = criterion(outputs, y)
            _, predicted = torch.max(outputs.data, 1)
            acc = (predicted == y).sum().item() / len(x)
        return loss.item(), acc*100


In [19]:
# Parâmetros do PSO
pop_size = 10
num_epochs = 150
inertia = 0.9
c1, c2 = 0.5, 0.9
# learning_rate = 0.0003
# beta1, beta2 = 0.5, 0.999
# epsilon = 1e-8

In [22]:
model = MLP(input_dim=34, output_dim=2)

particles = [Particle(model, device) for _ in range(pop_size)]
global_best_model = copy.deepcopy(particles[0].model)
global_best_score = float('inf')

criterion = nn.CrossEntropyLoss()

# m = {name: torch.zeros_like(param) for name, param in model.named_parameters()}
# v = {name: torch.zeros_like(param) for name, param in model.named_parameters()}

# Armazenar melhor acurácia
overall_global_best_accuracy = 0.0
overall_global_best_model = copy.deepcopy(global_best_model)

for epoch in range(num_epochs):
    # Ajustar inércia ao longo do tempo
    inertia = 0.9 - ((0.9 - 0.4)/num_epochs)*epoch

    for particle in particles:
        particle.model.train()
        particle.optimizer.zero_grad()

        # PSO+SGD
        # particle.calculate_grad(X_train, y_train, criterion)
        particle.pso(global_best_model, inertia, c1, c2)

        # Refinamento dos pesos com o Adam
        outputs = particle.model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        particle.optimizer.step()

        val_loss, val_acc = particle.evaluate_weights(X_train, y_train, criterion)

        # Avaliar e atualizar local best (com base no loss)
        if val_loss < particle.best_loss:
            particle.best_loss = val_loss
            particle.best_model = copy.deepcopy(particle.model)

    # Determinar g-best (menor perda)
    best_particle = min(particles, key=lambda p: p.best_loss)
    if best_particle.best_loss < global_best_score:
        global_best_score = best_particle.best_loss
        global_best_model = copy.deepcopy(best_particle.best_model)

    # Acurácia do g-best no teste
    test_loss, test_acc = best_particle.evaluate_weights(X_test, y_test, criterion)

    # Verificar se essa acurácia é a maior de todas.
    if test_acc > overall_global_best_accuracy:
        overall_global_best_accuracy = test_acc
        overall_global_best_model = copy.deepcopy(best_particle.best_model)

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {test_loss:.2f}, Acc: {test_acc:.2f}")

print("\nTreinamento concluído!")
print(f"Maior Acurácia Obtida: {overall_global_best_accuracy:.2f}%")

Epoch 10/150, Loss: 0.55, Acc: 87.74
Epoch 20/150, Loss: 1.46, Acc: 74.53
Epoch 30/150, Loss: 0.80, Acc: 81.13
Epoch 40/150, Loss: 0.88, Acc: 77.36
Epoch 50/150, Loss: 0.74, Acc: 83.02
Epoch 60/150, Loss: 0.77, Acc: 83.02
Epoch 70/150, Loss: 0.61, Acc: 83.02
Epoch 80/150, Loss: 0.54, Acc: 85.85
Epoch 90/150, Loss: 0.37, Acc: 89.62
Epoch 100/150, Loss: 0.36, Acc: 95.28
Epoch 110/150, Loss: 0.36, Acc: 95.28
Epoch 120/150, Loss: 0.34, Acc: 95.28
Epoch 130/150, Loss: 0.33, Acc: 95.28
Epoch 140/150, Loss: 0.33, Acc: 95.28
Epoch 150/150, Loss: 0.33, Acc: 95.28

Treinamento concluído!
Maior Acurácia Obtida: 95.28%
